[View in Colaboratory](https://colab.research.google.com/github/BisheshS/sentiment-analysis-yelp/blob/master/sentiment_only_cnn.ipynb)

In [0]:
import pandas as pd
import re
from nltk.stem.snowball import SnowballStemmer
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model, Sequential 
from keras.layers import Input, Activation
from keras.layers import LeakyReLU, BatchNormalization

from keras.layers import Dense
from keras.layers import Flatten
from keras import regularizers
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from nltk.stem import PorterStemmer

In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [4]:
df = pd.read_csv('/content/drive/Data/train.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
def clean_text(x):
    try:
        x = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", x)
        x = re.sub(r"\'s", " \'s", x)
        x = re.sub(r"\'ve", " \'ve", x)
        x = re.sub(r"n\'t", " n\'t", x)
        x = re.sub(r"\'re", " \'re", x)
        x = re.sub(r"\'d", " \'d", x)
        x = re.sub(r"\'ll", " \'ll", x)
        x = re.sub(r",", " , ", x)
        x = re.sub(r"!", " ! ", x)
        x = re.sub(r"\(", "", x)
        x = re.sub(r"\)", "", x)
        x = re.sub(r"\?", "", x)
        x = re.sub(r"/", "", x)
        x = re.sub(r"\s{2,}", " ", x)
        return x.lower()
    except:
        return None

In [0]:
df['text'] = df['text'].apply(clean_text)
df = df.dropna(how='any')
df['label'] = df['label'].apply(int)

In [0]:
# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer(num_words=2000)
	tokenizer.fit_on_texts(lines)
	return tokenizer

In [0]:
# calculate the maximum document length
def max_length(lines):
	return max([len(s.split()) for s in lines])


In [0]:
 
# encode a list of lines
def encode_text(tokenizer, lines, length):
	# integer encode
	encoded = tokenizer.texts_to_sequences(lines)
	# pad encoded sequences
	padded = pad_sequences(encoded, maxlen=length, padding='post')
	return padded

In [0]:
# load training dataset
trainLines = df['text'].values 
trainLabels = df['label'].values

In [11]:
# create tokenizer
tokenizer = create_tokenizer(trainLines)
# calculate max document length
length = max_length(trainLines)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)
trainX = encode_text(tokenizer, trainLines, length)
print(trainX.shape)
print(trainLabels)

Max document length: 1093
Vocabulary size: 186218
(400000, 1093)
[1 2 2 ... 2 2 0]


In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(trainX,trainLabels, test_size = 0.33, random_state = 42)
Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)

In [0]:
def define_model(lenght, vocal_size):
    model = Sequential()
    
    model.add(Embedding(vocab_size, 100, input_length=length))
    
    model.add(Conv1D(filters=256, kernel_size=4, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Conv1D(filters=256, kernel_size=4, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPooling1D())
    #model.add(Dropout(0.25))
    
    model.add(Conv1D(filters=128, kernel_size=4, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Conv1D(filters=128, kernel_size=4, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPooling1D())
    #model.add(Dropout(0.25))
    
    model.add(Conv1D(filters=64, kernel_size=4, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Conv1D(filters=64, kernel_size=4, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPooling1D())
    model.add(Dropout(0.5))
    
    model.add(Flatten())
    
    model.add(Dense(10, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	# summarize
    print(model.summary())
	#plot_model(model, show_shapes=True, to_file='multichannel.png')
    return model

In [20]:
# define model
model = define_model(length, 3000)
# fit model
#model.fit([X_train,X_train,X_train], array(Y_train), epochs=10, batch_size=200)
model.fit(X_train, array(Y_train), validation_data=(X_test, Y_test), epochs=5, batch_size=200)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 1093, 100)         18621800  
_________________________________________________________________
conv1d_21 (Conv1D)           (None, 1093, 256)         102656    
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 1093, 256)         0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 1093, 256)         1024      
_________________________________________________________________
conv1d_22 (Conv1D)           (None, 1093, 256)         262400    
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 1093, 256)         0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 1093, 256)         1024      
__________

267800/268000 [============================>.] - ETA: 0s - loss: 0.2254 - acc: 0.9046

268000/268000 [==============================] - 1150s 4ms/step - loss: 0.2254 - acc: 0.9046 - val_loss: 0.1751 - val_acc: 0.9318
Epoch 2/5
 63400/268000 [======>.......................] - ETA: 12:45 - loss: 0.1565 - acc: 0.9376

268000/268000 [==============================] - 1148s 4ms/step - loss: 0.1565 - acc: 0.9379 - val_loss: 1.2554 - val_acc: 0.8543
Epoch 3/5
  9200/268000 [>.............................] - ETA: 16:08 - loss: 0.1420 - acc: 0.9455

267800/268000 [============================>.] - ETA: 0s - loss: 0.1421 - acc: 0.9443

268000/268000 [==============================] - 1147s 4ms/step - loss: 0.1421 - acc: 0.9443 - val_loss: 0.2501 - val_acc: 0.9130
Epoch 4/5
 63400/268000 [======>.......................] - ETA: 12:45 - loss: 0.1276 - acc: 0.9502

268000/268000 [==============================] - 1147s 4ms/step - loss: 0.1321 - acc: 0.9480 - val_loss: 0.1763 - val_acc: 0.9322
Epoch 5/5
  9200/268000 [>.............................] - ETA: 16:06 - loss: 0.1177 - acc: 0.9567

267800/268000 [============================>.] - ETA: 0s - loss: 0.1229 - acc: 0.9526

268000/268000 [==============================] - 1146s 4ms/step - loss: 0.1229 - acc: 0.9525 - val_loss: 0.1603 - val_acc: 0.9392


In [22]:
scores = model.evaluate(X_test, Y_test, verbose=0)
print("CNN ACC: %.2f%%" % (scores[1]*100))

CNN ACC: 93.92%


In [0]:
model.save('/content/drive/Data/model_moring.h5')

In [0]:
tokenizer.dump('/content/drive/Data/tokenizer.h5')

AttributeError: ignored

In [0]:
import pickle

# saving
with open('/content/drive/Data/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)